In [ ]:
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

# Step 1: Load the existing model
model = load_model('age_gender_model.h5', custom_objects={'mae': MeanAbsoluteError()})
X = np.load('/content/drive/MyDrive/X.npy')
y_gender = np.load('/content/drive/MyDrive/y_gender.npy')
y_age = np.load('/content/drive/MyDrive/y_age.npy')


# Step 2: Define the improved model architecture
input_shape = model.input_shape[1:]  # Skip the batch size dimension

inputs = Input(shape=input_shape)

# Convolutional layers with batch normalization
conv_1 = Conv2D(32, (3, 3), activation='relu')(inputs)
batch_norm_1 = BatchNormalization()(conv_1)
maxp_1 = MaxPooling2D(pool_size=(2, 2))(batch_norm_1)

conv_2 = Conv2D(64, (3, 3), activation='relu')(maxp_1)
batch_norm_2 = BatchNormalization()(conv_2)
maxp_2 = MaxPooling2D(pool_size=(2, 2))(batch_norm_2)

conv_3 = Conv2D(128, (3, 3), activation='relu')(maxp_2)
batch_norm_3 = BatchNormalization()(conv_3)
maxp_3 = MaxPooling2D(pool_size=(2, 2))(batch_norm_3)

conv_4 = Conv2D(256, (3, 3), activation='relu')(maxp_3)
batch_norm_4 = BatchNormalization()(conv_4)
maxp_4 = MaxPooling2D(pool_size=(2, 2))(batch_norm_4)

flatten = Flatten()(maxp_4)

# Fully connected layers with dropout
dense_1 = Dense(256, activation='relu')(flatten)
dropout_1 = Dropout(0.3)(dense_1)

dense_2 = Dense(256, activation='relu')(dropout_1)
dropout_2 = Dropout(0.3)(dense_2)

# Output layers
output_1 = Dense(1, activation='sigmoid', name='gender_out')(dropout_2)  # Gender prediction
output_2 = Dense(1, activation='relu', name='age_out')(dropout_2)        # Age prediction

# Define the improved model
improved_model = Model(inputs=inputs, outputs=[output_1, output_2])

# Compile the improved model
improved_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=['binary_crossentropy', 'mae'],
    metrics=[['accuracy'], ['mae']]
)

# Define early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor the validation loss
    patience=10,         # Number of epochs to wait before stopping if no improvement
    restore_best_weights=True  # Restore the weights from the epoch with the best validation loss
)

# Print the summary of the improved model
improved_model.summary()

from tensorflow.keras.utils import Sequence

class DataGenerator(Sequence):
    def __init__(self, X, y_gender, y_age, batch_size, subset, datagen):
        self.X = X
        self.y_gender = y_gender
        self.y_age = y_age
        self.batch_size = batch_size
        self.datagen = datagen
        self.indexes = np.arange(len(self.X))
        self.subset = subset
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.X) / self.batch_size))

    def __getitem__(self, index):
        batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        batch_X = self.X[batch_indexes]
        batch_y_gender = self.y_gender[batch_indexes]
        batch_y_age = self.y_age[batch_indexes]

        batch_X_augmented = self.datagen.flow(batch_X, batch_size=self.batch_size, shuffle=False)

        return next(batch_X_augmented), (batch_y_gender, batch_y_age)

    def on_epoch_end(self):
        if self.subset == 'training':
            np.random.shuffle(self.indexes)

# Create data generators
train_datagen = ImageDataGenerator(
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = DataGenerator(X, y_gender, y_age, batch_size=64, subset='training', datagen=train_datagen)
validation_generator = DataGenerator(X, y_gender, y_age, batch_size=64, subset='validation', datagen=train_datagen)

# Train the model
history = improved_model.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)

# Save the improved model
improved_model.save('improved_age_gender_model_with_augmentation.h5')


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 128, 128, 1)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 126, 126, 32)   │            320 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 126, 126, 32)   │            128 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_4           │ (None, 63, 63, 32)     │              0 │ batch_normalization_4… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 61, 61, 64)     │         18,496 │ max_pooling2d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_5     │ (None, 61, 61, 64)     │            256 │ conv2d_5[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_5           │ (None, 30, 30, 64)     │              0 │ batch_normalization_5… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 28, 28, 128)    │         73,856 │ max_pooling2d_5[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_6     │ (None, 28, 28, 128)    │            512 │ conv2d_6[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_6           │ (None, 14, 14, 128)    │              0 │ batch_normalization_6… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 12, 12, 256)    │        295,168 │ max_pooling2d_6[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_7     │ (None, 12, 12, 256)    │          1,024 │ conv2d_7[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_7           │ (None, 6, 6, 256)      │              0 │ batch_normalization_7… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_1 (Flatten)       │ (None, 9216)           │              0 │ max_pooling2d_7[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 256)            │      2,359,552 │ flatten_1[0][0]        │
├──────────────────────

 Total params: 2,815,618 (10.74 MB)

 Trainable params: 2,814,658 (10.74 MB)

 Non-trainable params: 960 (3.75 KB)

Epoch 1/100
370/370 ━━━━━━━━━━━━━━━━━━━━ 88s 214ms/step - age_out_mae: 15.0323 - gender_out_accuracy: 0.5700 - loss: 16.3377 - val_age_out_mae: 21.0517 - val_gender_out_accuracy: 0.5240 - val_loss: 22.0155
Epoch 2/100
370/370 ━━━━━━━━━━━━━━━━━━━━ 78s 208ms/step - age_out_mae: 11.6700 - gender_out_accuracy: 0.6187 - loss: 12.3592 - val_age_out_mae: 12.2259 - val_gender_out_accuracy: 0.6489 - val_loss: 12.8537
Epoch 3/100
370/370 ━━━━━━━━━━━━━━━━━━━━ 78s 209ms/step - age_out_mae: 10.4550 - gender_out_accuracy: 0.6246 - loss: 11.1181 - val_age_out_mae: 19.1103 - val_gender_out_accuracy: 0.6568 - val_loss: 19.7400
Epoch 4/100
370/370 ━━━━━━━━━━━━━━━━━━━━ 80s 204ms/step - age_out_mae: 9.8887 - gender_out_accuracy: 0.6412 - loss: 10.5377 - val_age_out_mae: 10.8187 - val_gender_out_accuracy: 0.6571 - val_loss: 11.4433
Epoch 5/100
370/370 ━━━━━━━━━━━━━━━━━━━━ 79s 213ms/step - age_out_mae: 9.3782 - gender_out_accuracy: 0.6399 - loss: 10.0219 - val_age_out_mae: 15.6002 - val_gender_out_accuracy:

In [ ]:
# Evaluate the model on the validation set
val_loss, val_gender_loss, val_age_loss = improved_model.evaluate(validation_generator)

# Print the evaluation results
print(f"Validation Loss: {val_loss}")
print(f"Validation Gender Loss: {val_gender_loss}")
print(f"Validation Age Loss: {val_age_loss}")

# Access the metrics (accuracy and mae) from the history object
print(f"Validation Gender Accuracy: {history.history['gender_out_accuracy'][-1]}")
print(f"Validation Age MAE: {history.history['age_out_mae'][-1]}")

370/370 ━━━━━━━━━━━━━━━━━━━━ 38s 103ms/step - age_out_mae: 11.5615 - gender_out_accuracy: 0.8133 - loss: 11.9547
Validation Loss: 8.248579025268555
Validation Gender Loss: 7.901363372802734
Validation Age Loss: 0.8354730010032654
Validation Gender Accuracy: 0.8304898738861084
Validation Age MAE: 6.699906826019287
